In [ ]:
#TODO remove
interesting code:
#: https://github.com/reeanne/FinalProject/blob/cef6b45060ad9646f2be4de93c226aebeadec41b/essentia-master/src/examples/python/streaming_extractor/tonaldescriptors.py
#https://github.com/LqNoob/Essentia/blob/7a70a25dd6668855b3677bd0cae0df190e319cbf/test/src/unittest/tonal/test_chordsdetection_streaming.py
#∫imple one https://github.com/mariogearth/ChordsDetectionPython/blob/b69e813bef92a361e7cad0a58bbd0f049b252914/old%20stuff/chords_test3.py

In [ ]:
#check student notebook later for loadChromaAndSpectrum 

In [1]:
import sys, os
import essentia, essentia.standard, essentia.streaming
from essentia.streaming import *

In [5]:
def computeChords(filename, frameSize=4096, hopSize=2048, tuningFrequency=440.0):
    
    pool = essentia.Pool()
    loader = essentia.streaming.MonoLoader(filename=filename)

    #loader = esstr.MonoLoader(filename = filename)
    #framecutter = esstr.FrameCutter(hopSize=hopSize, frameSize=frameSize)
    
    #get the frames and compute chord detection
    fc = FrameCutter(frameSize=frameSize,
                     hopSize=hopSize,
                     silentFrames='noise')

    w = Windowing(type='blackmanharris62')
    spec = Spectrum()
    spectralpeaks = SpectralPeaks(orderBy="magnitude",
                                      magnitudeThreshold=1e-05,
                                      minFrequency=40,
                                      maxFrequency=5000,
                                      maxPeaks=10000)
    
    #TODO: https://essentia.upf.edu/documentation/reference/std_HPCP.html
    hpcp = HPCP(
        size=12,
        referenceFrequency = tuningFrequency,
        harmonics = 8,
        bandPreset = True,
        minFrequency = 40.0,
        maxFrequency = 5000.0,
        bandSplitFrequency = 250.0,
        weightType = "cosine",
        nonLinear = False,
        windowSize = 1.0,
        normalized='unitSum')
    
    #TODO: check parameters https://essentia.upf.edu/documentation/reference/std_ChordsDetection.html
    chords = ChordsDetection()
    chords_desc = ChordsDescriptors()


    # connect algorithms together
    loader.audio >> fc.signal
    fc.frame >> w.frame >> spec.frame
    spec.spectrum >> spectralpeaks.spectrum
    spec.spectrum >> (pool, 'spectrum.magnitude') #mine
    spectralpeaks.frequencies >> hpcp.frequencies
    spectralpeaks.magnitudes >> hpcp.magnitudes
    hpcp.hpcp >> (pool, 'chroma.hpcp')  #mine
    hpcp.hpcp >> chords.pcp
    chords.chords >> (pool, 'chords.chords')
    chords.strength >> (pool, 'chords.strength')
    #hpcp_key.hpcp >> key.pcp
    #peaks.frequencies >> hpcp_chord.frequencies
    #peaks.magnitudes >> hpcp_chord.magnitudes
    #hpcp_chord.hpcp >> chords.pcp
    #chords.chords >> chords_desc.chords
    #key.key >> chords_desc.key
    #key.scale >> chords_desc.scale
    #peaks.frequencies >> hpcp_tuning.frequencies
    #peaks.magnitudes >> hpcp_tuning.magnitudes
    
    
    
    #hpcp_key.hpcp >>(pool, 'chroma.hpcp')


    
    #TODO:check pools https://essentia.upf.edu/documentation/essentia_python_tutorial.html 
    essentia.run(loader)

    
    test= pool['spectrum.magnitude']
    test2= pool['chroma.hpcp']
    test3= pool['chords.chords']
    test4= pool['chords.strength']
    
    return test, test2, test3, test4

In [ ]:
loader

In [6]:
testAudio = './test-audio/Disc 1 - 01 - Maple Leaf Rag.flac'
(test, test2, test3, test4) =computeChords(testAudio)

In [12]:
test.shape


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
0.00001